### Handle course data update

#### Fetch data

In [1]:
from scraper import CourseScraper

cs = CourseScraper(save_captchas=False)
timestamp = cs.parse_all(skip_parsed=True, manual=False, verbose=False)
cs.post_processing(stat=True)
cs.info()

Parsing courses for all 251 subjects, skip if already existed


100%|██████████| 251/251 [00:17<00:00, 14.30it/s]


Done! Saved at data/1661771599
Found 2819 instructors
Number of departments: 125
Number of courses: 8413


#### Update cutopia-data

In [2]:
import os

# timestamp = 1661771599
p = os.system(f'sh move_data.sh {timestamp}')


moving 1661771599 to /home/mike/Github/CUtopia/data


+ git add .
+ git commit -am Update data @1661771599


[master e67d48d] Update data @1661771599
 36 files changed, 36 insertions(+), 36 deletions(-)
 rewrite courses/ACCT.json (93%)
 rewrite courses/CCSS.json (71%)
 rewrite courses/CHEM.json (75%)
 rewrite courses/CHES.json (90%)
 rewrite courses/COMM.json (83%)
 rewrite courses/CSCI.json (76%)
 rewrite courses/DSME.json (77%)
 rewrite courses/DSPS.json (66%)
 rewrite courses/EDUC.json (70%)
 rewrite courses/ELTU.json (87%)
 rewrite courses/ENGE.json (82%)
 rewrite courses/ESTR.json (80%)
 rewrite courses/GRON.json (86%)
 rewrite courses/LAWS.json (82%)
 rewrite courses/MEDU.json (88%)
 rewrite courses/MUSC.json (62%)
 rewrite courses/RMSC.json (76%)
 rewrite courses/SPED.json (67%)
 rewrite courses/TRAN.json (89%)
 rewrite courses/UGEC.json (63%)
 rewrite courses/UGED.json (85%)
 rewrite resources/instructors.json (98%)


+ git push origin master
To https://github.com/cutility/CUtopia-data.git
   37205ee..e67d48d  master -> master


#### Update FE & BE

In [25]:
import time
from dotenv import set_key

deploy_duration = 15 # in mins (need to deploy both fe & be within the duration)
deploy_duration_seconds = deploy_duration * 60 # in mins

now = str(int((time.time() + deploy_duration_seconds) * 1000))

# Update FE
set_key('../frontend/.env', 'REACT_APP_LAST_DATA_UPDATE', str(now))

# Update BE (TODO)

print(f'Updated, please deploy in {deploy_duration} mins')

Updated, please deploy in 15 mins
